# Markov Model - Text Classifier

Starting from 2 sets of poems by 2 different authors: Edgar Allan Poe and Robert Frost, build a text classifier that can distinguished between the 2 authors.
 - Compute train and test accuracy
 - Check for class imbalance, compute F1-score if imbalanced

### Outline of the code:
 - Loop through each file, save each line to a list (one line == one sample)
 - Save the labels too
 - train-test split
 - Create a mapping from unique word to unique int index
    - loop through data and tokenize each line (string split is enough)
    - Assign each unique word a unique index
    - create a special index for unknown word (words that could be in test set but are not in training set)  
 - Convert each line of text into integer lists
 - Train a Markov model for each class (Edgar Allan Poe / Robert Frost)
 - Use smoothing (add-one smoothing)
 - Do we need A and pi or just Log(A) and Log(pi)?
 - We also need to compute the priors p(class = k) to know if we need to take it into account in Baye's rule.
 - Write a function to compute the posterior for each class, given an input
 - Take the argmax over the posteriors to get the predicted class
 - Make predictions for both train and test sets
 - Compute accuracy for train and test sets
 - Check for class imbalance
 - Check confusion matrix and F1-score

In [80]:
#get datasets
#!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
#!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

### Dataset creation

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import string
import pandas as pd
from sklearn.model_selection import train_test_split

In [82]:
input_files = [
  '../datasets/poems/edgar_allan_poe.txt',
  '../datasets/poems/robert_frost.txt',
]
# label: Edgar Allan Poe => 0, Robert Frost => 1

In [83]:
inputs = []
labels = []
for i, filepath in enumerate(input_files):
    with open(filepath) as file:
      for line in file:
         line = line.rstrip().lower() # remove trailing white space and convert to lower case
         if line:
          #remove punctuation -- try with or without, it's not a clear change on the test set f1-score
          line = line.translate(str.maketrans('','',string.punctuation))
          inputs.append(line)
          labels.append(i)

In [84]:
inputs_train, inputs_test, Ytrain, Ytest = train_test_split(inputs, labels, random_state=123)

### Create a mapping from each word to a unique index

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizer = CountVectorizer(tokenizer=(lambda x: x.lower().split(' ')))

In [86]:
Xtrain = vectorizer.fit_transform(inputs_train)
Xtest = vectorizer.transform(inputs_test)

/Users/clement/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [87]:
#Get the vocabulary dict from vectorizer
word2idx = vectorizer.vocabulary_
idx2word = {v: k for k, v in word2idx.items()}

In [88]:
# create a special index for "unknown" word
idx2word[len(word2idx)] = "unk"
word2idx["unk"] = len(word2idx)

In [89]:
vocab_size = len(word2idx)
print("vocabulary size: ", vocab_size)

vocabulary size:  2548


### Train a Markov model A0 and pi0 for Edgar Allan Poe, and another model A1 and pi1 for Robert Frost

Let's compute pi0 first.  
pi0 is a vector of size vocab_size, and each component i is the total count of word i divided by total count of all words.  
So for all Xtrain[k,:] where k such as Y[k] == 0, pi0[i] = np.sum(Xtrain_k[:,i])/np.sum(Xtrain_k)

In [90]:
pi0 = np.ones(vocab_size)
pi1 = np.ones(vocab_size)
sum_Xtrain_0 = vocab_size # with Add-One smoothing
sum_Xtrain_1 = vocab_size
for j in range(Xtrain.shape[1]):
  for i in range(Xtrain.shape[0]):
    if Ytrain[i] == 0:
        # compute pi0
        pi0[j] += Xtrain[i,j]
        sum_Xtrain_0 += Xtrain[i,j]
    if Ytrain[i] == 1:
        # compute pi1
        pi1[j] += Xtrain[i,j]
        sum_Xtrain_1 += Xtrain[i,j]
pi0 = np.log(pi0) - np.log(sum_Xtrain_0)
pi1 = np.log(pi1) - np.log(sum_Xtrain_1)

In [91]:
# The word with the highest initial probability for Edgar Allan Poe is:
idx2word[np.argmax(pi0)]

'the'

Let's compute A0 and A1 now.  
With Add-One smoothing, Aij = (count(word i to word j) + 1) / (count(word i) + vocab_size)
In order to compute count(word i to word j), we first need to transfrom our list of words into list of int using word2idx


In [92]:
inputs_train[0].split(' ')

['mother', 'of', 'god', 'be', 'with', 'me', 'still']

In [93]:
list(map(lambda x: word2idx[x],inputs_train[0].lower().split(' ')))

[1348, 1429, 841, 141, 2480, 1283, 2020]

In [94]:
inputs_train_index = []
A0 = np.ones([vocab_size, vocab_size]) # we start at 1 for Add-One smoothing
A1 = np.ones([vocab_size, vocab_size])
for i in range(len(inputs_train)):
  # tokenize then transform into list of int
  inputs_train_index.append(list(map(lambda x: word2idx[x],inputs_train[i].lower().split(' '))))
  for j in range(len(inputs_train_index[i])-1):
    if Ytrain[i] == 0:
      # compute A0
      A0[inputs_train_index[i][j],inputs_train_index[i][j+1]]+=1
    if Ytrain[i] == 1:
      # compute A1
      A1[inputs_train_index[i][j],inputs_train_index[i][j+1]]+=1



In [95]:
A0

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [96]:
# we still need to divide by count(word i) + vocab_size
indices_label_0 = [i for i in range(len(Ytrain)) if Ytrain[i] == 0]
indices_label_1 = [i for i in range(len(Ytrain)) if Ytrain[i] == 1]
    

In [97]:
word_counts_0 = np.ones(vocab_size)*vocab_size
word_counts_1 = np.ones(vocab_size)*vocab_size

for j in range(vocab_size-1):
  for i in indices_label_0:
    word_counts_0[j] += Xtrain[i,j]
  for i in indices_label_1:
    word_counts_1[j] += Xtrain[i,j]

In [98]:
print(word_counts_0)
print(A0[0,:])

[2595. 2620. 2548. ... 2548. 2549. 2548.]
[1. 1. 1. ... 1. 1. 1.]


Now we need to divide A0[i,:] and A1[i,:] by word_counts0[i] and word_count1[i] respectively.  
We can also store A0 and A1 as log(A0) and log(A1). Same for pi0 and pi1

In [99]:
for i in range(vocab_size):
    A0[i,:] = np.log(A0[i,:]) - np.log(word_counts_0[i])
    A1[i,:] = np.log(A1[i,:]) - np.log(word_counts_1[i])
    

In [100]:
A0

array([[-7.8613418 , -7.8613418 , -7.8613418 , ..., -7.8613418 ,
        -7.8613418 , -7.8613418 ],
       [-7.8709296 , -7.8709296 , -7.8709296 , ..., -7.8709296 ,
        -7.8709296 , -7.8709296 ],
       [-7.84306402, -7.84306402, -7.84306402, ..., -7.84306402,
        -7.84306402, -7.84306402],
       ...,
       [-7.84306402, -7.84306402, -7.84306402, ..., -7.84306402,
        -7.84306402, -7.84306402],
       [-7.8434564 , -7.8434564 , -7.8434564 , ..., -7.8434564 ,
        -7.8434564 , -7.8434564 ],
       [-7.84306402, -7.84306402, -7.84306402, ..., -7.84306402,
        -7.84306402, -7.84306402]])

Now let's compute P(k = 0) and P(k =1)

In [101]:
Pk1 = np.sum(Ytrain) / len(Ytrain)
Pk0 = 1-Pk1
print("Pobability of having a poem from Edgar Allan Poe in the training set: ", Pk0, " Probablity of Robert Frost: ", Pk1)
logPk0 = np.log(Pk0)
logPk1 = np.log(Pk1)
print(logPk0, logPk1)

Pobability of having a poem from Edgar Allan Poe in the training set:  0.3331269349845202  Probablity of Robert Frost:  0.6668730650154798
-1.0992316754949722 -0.40515555850036844


## Let's build the classifier now!

So if we call x a poem, and k the class author with k = 0 for Edgar Allan Poe and k = 1 for Robert Frost.  
We are looking for the estimator k* = argmax_k( p(class = k ) | x )  i.e the maximum of the probabilities of authors given and poem.  

Bayes' Rule says:
p(author | poem) = p(poem | author) * p(author) / p(poem)  

Since we want the argmax, we can simplify the expression above by removing the denominator p(poem). Indeed p(poem) is a constant in our dataset and does not depend on author, so it can be removed. So now we are looking for:

k* = argmax( p(author | poem) ) = argmax( p(poem | author) * p(author) )

We can take the log of the expression above, as the log function is monotonously increasing, it does not change the order of the argmax.  

k* = argmax_k( log(p(poem | author = k)) + log(p(author = k)) ) = argmax_k( log(p(x | k)) + log(p(k)) )  

We just calculated p(k) above. And each Markov model gives us p(x | k).  

Indeed, each poem is a list of words. We use our Markov models A0, pi0 and A1, pi1 to compute the probabilty of this sequence of word for each author. The highest probability is our estimate. That is p( x | k ).  

So for a poem x of size N:  

log(p( x | k = 0)) = log(pi0[x[0]]) + sum_on_i_to_N( log(A0[x[i],x[i+1]])) 





In [102]:
def text_classifier(x):
    # compute p(x | k = 0) and p(x | k=1)
    logPx_k0 = pi0[x[0]]
    logPx_k1 = pi1[x[0]]
    for i in range(len(x)-1):
        logPx_k0 += A0[x[i],x[i+1]]
        logPx_k1 += A1[x[i],x[i+1]]
    # add log( p(k = author) ) to get the full log(p(k=author | x))
    log_p0_given_x = logPx_k0 + logPk0
    log_p1_given_x = logPx_k1 + logPk1
    return 0 if log_p0_given_x > log_p1_given_x else 1

In [103]:
print(text_classifier(inputs_train_index[2])) # Ytrain[2] == 0
print(text_classifier(inputs_train_index[3])) # Ytrain[3] == 1

0
1


Convert input_test into a list of word indexes

In [104]:
def map_words_to_indexes(word):
  try:
    return word2idx[word]
  except Exception:
    return word2idx['unk']

inputs_test_index = []
for i in range(len(inputs_test)):
  inputs_test_index.append(list(map(lambda x: map_words_to_indexes(x), inputs_test[i].lower().split(' '))))

In [105]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [106]:
def text_inputs_classifier(X):
    Y = np.zeros(len(X))
    for i in range(len(X)):
        Y[i] = text_classifier(X[i])
    return Y

In [107]:
Y_train_pred = text_inputs_classifier(inputs_train_index)
Y_test_pred = text_inputs_classifier(inputs_test_index)

In [108]:
training_accuracy = accuracy_score(Ytrain, Y_train_pred)
test_accuracy = accuracy_score(Ytest, Y_test_pred)
training_f1_score = f1_score(Ytrain, Y_train_pred)
test_f1_score = f1_score(Ytest, Y_test_pred)
cm_train = confusion_matrix(Ytrain, Y_train_pred)
cm_test = confusion_matrix(Ytest, Y_test_pred)

print("training accuracy: ", training_accuracy)
print("test accuracy: ", test_accuracy)
print("training f1-score: ", training_f1_score)
print("test f1-score: ", test_f1_score)
print("training confusion matrix: ", cm_train)
print("test confusion matrix: ", cm_test)

training accuracy:  0.9956656346749226
test accuracy:  0.8200371057513914
training f1-score:  0.996760758907913
test f1-score:  0.8792029887920298
training confusion matrix:  [[ 531    7]
 [   0 1077]]
test confusion matrix:  [[ 89  91]
 [  6 353]]
